In [1]:
import os
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'))
langsmith_key = os.getenv('LANGSMITH_API_KEY')
langsmith_trace = os.getenv('LANGCHAIN_TRACING_V2')




In [2]:
from langchain.tools import tool



from langchain_core.tools import tool

@tool
def sql_tool(query):

    """
    Generates SQL queries based on the user's input question.
    If the query is unclear, asks for more information.
    """
    prompt = ChatPromptTemplate.from_messages(
            [
                ("system","you are an helpfull assistent to answer the user SQL releated questions & making SQL Queries.if dont know answer say to use provoide more informationv "),
                ("human", "{input}"),

            ]
        )

    chain = prompt | llm
        
    response = chain.invoke({"input": query})

    return response.content

@tool
def marketing_tool(query):
    """
    
    Generates marketing strategies based on the user's input question.
    If the query is unclear, asks for more information.
     """

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent to answer the user company marketing releated questions using advice they can improve their business.if dont know answer say to use provoide more informationv "),
            ("human", "{input}"),

        ]
    )

    chain = prompt | llm
    
    response = chain.invoke({"input": query})
    
    return response.content
    


In [ ]:

tools = [sql_tool,marketing_tool]

agent_excutor = create_react_agent(llm, tools)


In [25]:
# Run Agent
prompt = "How to speak english with in 30 days?"
response = agent_excutor.invoke({"messages": prompt})

In [26]:
response

{'messages': [HumanMessage(content='How to speak english with in 30 days?', additional_kwargs={}, response_metadata={}, id='08325582-aa12-4035-8064-d6d0d904432e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FT1sqysz6EExzj32XaLjJzTG', 'function': {'arguments': '{"query": "How to learn English in 30 days?"}', 'name': 'marketing_tool'}, 'type': 'function'}, {'id': 'call_qddnjFq2fPpZVJFtB5D9HGMu', 'function': {'arguments': '{"query": "How to improve English speaking skills in 30 days?"}', 'name': 'sql_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 109, 'total_tokens': 171, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}

In [38]:
prompt = "Generate SQL Query top 10 records retrive query?"
response = agent_excutor.invoke({"messages": prompt})

In [7]:
from langchain_core.messages import HumanMessage


In [47]:
response['messages']

[HumanMessage(content='Generate SQL Query top 10 records retrive query?', additional_kwargs={}, response_metadata={}, id='8003ad67-6ec3-4970-8621-54873a4e2ee4'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_L4p5DFjul0k0xPppW8xdL8zi', 'function': {'arguments': '{"query":"SELECT * FROM table_name LIMIT 10;"}', 'name': 'sql_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 110, 'total_tokens': 132, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-53c2b814-9494-4a6a-aaa2-243adadc40f0-0', tool_calls=[{'name': 'sql_tool', 'args': {'query': 'SELECT * FROM table_name LIMIT 10;'}, 'id': 'call_L4p5DFjul0k0xPppW8xdL8zi', 'ty

In [49]:
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_excutor.stream(
    {"messages": [HumanMessage(content = prompt)]},config=config):
    print(chunk)
    print("--------------------------------")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TvQJn3p9e8zGLTKRO67eipJQ', 'function': {'arguments': '{"query":"SELECT * FROM table_name LIMIT 10;"}', 'name': 'sql_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 110, 'total_tokens': 132, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7d898d48-d5ff-46ca-a038-caeb9bf49668-0', tool_calls=[{'name': 'sql_tool', 'args': {'query': 'SELECT * FROM table_name LIMIT 10;'}, 'id': 'call_TvQJn3p9e8zGLTKRO67eipJQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 110, 'output_tokens': 22, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cac

# Memory Saver

In [4]:
# Adding Memory using Langgraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [5]:
agent_excutor = create_react_agent(llm , tools , checkpointer= memory)

In [8]:
# config = { "configurable" : { "thread_id" : "kp123"} }
# prompt = "Give me top records retrive query?"
# for chunk in agent_excutor.stream({"messages":[HumanMessage(content=prompt)]},config= config):
#     print(chunk)
#     print("--------------------------------")

config = {"configurable": {"thread_id": "kp123"}}
prompt = "Give me top records retrieve query?"

for chunk in agent_excutor.stream(
    {"messages": [HumanMessage(content=prompt)]}, config=config
):
    print(chunk)
    print("--------------------------------")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cG3ig0srjbrPRNoDKpnWkWvK', 'function': {'arguments': '{"query":"Retrieve the top records from the database."}', 'name': 'sql_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 106, 'total_tokens': 127, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-27d847b6-7691-47a7-9136-a6aa4c7b8d99-0', tool_calls=[{'name': 'sql_tool', 'args': {'query': 'Retrieve the top records from the database.'}, 'id': 'call_cG3ig0srjbrPRNoDKpnWkWvK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 106, 'output_tokens': 21, 'total_tokens': 127, 'input_token_details':

In [9]:
config = {"configurable": {"thread_id": "kp123"}}
prompt = "Please explain that query so that i can also understand it right?"

for chunk in agent_excutor.stream(
    {"messages": [HumanMessage(content=prompt)]}, config=config
):
    print(chunk)
    print("--------------------------------")

{'agent': {'messages': [AIMessage(content='Sure! Let me break down the SQL query for you:\n\n1. `SELECT *`: This part of the query specifies which columns to retrieve from the database. In this case, `*` is a wildcard that indicates all columns should be selected.\n\n2. `FROM your_table`: This part specifies the table from which to retrieve the records. You need to replace `your_table` with the actual name of the table you want to query.\n\n3. `ORDER BY your_column`: This part of the query orders the results based on a specific column. You need to replace `your_column` with the column name you want to use for sorting.\n\n4. `LIMIT 10`: This part limits the number of records returned by the query. In this case, it limits the result set to the top 10 records based on the ordering specified in the `ORDER BY` clause.\n\nSo, when you run this query, it will retrieve all columns from the specified table, order the records based on a specific column, and then return only the top 10 records fr

In [13]:
prompt = "show me the best marking strategy for sellingphones"

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_excutor.stream(
    {"messages": [HumanMessage(content = prompt)]},config=config):
    print(chunk)
    print("--------------------------------")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hiByNWFU88KbfSN2e99e8ThE', 'function': {'arguments': '{"query":"best marketing strategy for selling phones"}', 'name': 'marketing_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 279, 'total_tokens': 299, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c689ab75-e65c-48bc-9093-65e4a3002799-0', tool_calls=[{'name': 'marketing_tool', 'args': {'query': 'best marketing strategy for selling phones'}, 'id': 'call_hiByNWFU88KbfSN2e99e8ThE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 20, 'total_tokens': 299, 'input_token